In [1]:
from underthesea import pos_tag
import re
import string
import csv
import pandas as pd
import numpy as np

In [2]:
def preprosTrain():
    dic = {}
    labels = {}
    with open('scrapy_test/spiders/Output/labels.txt', 'r', encoding='utf-8') as f:
        for line in f.readlines():
            labels[line.strip()] = len(labels)
            
    '''with open('scrapy_test/spiders/Output/_trainSet.txt', 'r', encoding='utf-8') as f:
        for para in f.readlines():
            category, para = para.split('\t', 1)
            for s in pos_tag(para):
                if s[1] in ('A', 'V') or re.search('^N', s[1]):
                    w = re.sub(r'[' + string.punctuation + ']', ' ', s[0]).strip().lower()
                    if w != '':
                        if w not in dic:
                            dic[w] = [0] * len(labels)
                        dic[w][labels[category]] += 1'''
                        
    with open('train.txt', 'r', encoding='utf-8') as f:
        for para in f.readlines():
            category, para = para.split('\t', 1)
            for s in pos_tag(para):
                if s[1] in ('A', 'V') or re.search('^N', s[1]):
                    w = re.sub(r'[' + string.punctuation + ']', ' ', s[0]).strip().lower()
                    if w != '':
                        if w not in dic:
                            dic[w] = [0] * len(labels)
                        dic[w][labels[category]] += 1

    df = pd.DataFrame(dic, index=labels.keys())
    df.to_csv('featureVectors.csv')

In [3]:
labelWordCnt = {}

def train():
    with open('scrapy_test/spiders/Output/labels.txt', 'r', encoding='utf-8') as f:
        for line in f.readlines():
            labelWordCnt[line.strip()] = 0
    
    global df
    df = pd.read_csv('featureVectors.csv').set_index('Unnamed: 0')
    
    # chuan hoa vector theo cot
    for w in df:
        mx = df[w].max()
        mn = df[w].min()
        df[w] = (df[w] - mn) / (mx - mn)
    
    # tinh p(c), p(wj|c)
    totalWordCnt = np.sum(df.sum())
    for gr in df.index:
        sumOfGroup = df.loc[gr].sum()
        df.loc[gr] = (df.loc[gr] + 0.1) / (sumOfGroup + len(df.columns)/10)
        labelWordCnt[gr] = sumOfGroup / totalWordCnt

In [4]:
preprosTrain()

In [5]:
import math
def test():
    global df
    ans = []
    with open('test.txt', 'r', encoding='utf-8') as f:
        for para in f.readlines():
            prob = {}
            for typ in labelWordCnt:
                if labelWordCnt[typ] == 0.0:
                    prob[typ] = -100000000
                else:
                    prob[typ] = math.log(labelWordCnt[typ], 2)
            
            for s in pos_tag(para):
                if s[1] in ('A', 'V') or re.search('^N', s[1]):
                    w = re.sub(r'[' + string.punctuation + ']', ' ', s[0]).strip().lower()
                    if w != '':
                        if w not in df:
                            continue
                        for typ in labelWordCnt:
                            prob[typ] += math.log(df.loc[typ, w], 2)
            
            prob = sorted(prob.items(), key=lambda x:x[1], reverse=True)[0:2]
            ans.append([prob[0][0], prob[1][0]])
    
    with open('testAns.txt', 'r', encoding='utf-8') as f:
        total, correct1, correct2 = 0, 0, 0
        for key in f.readlines():
            if total == len(ans):
                break
            if key.strip() == ans[total][0]:
                correct1 += 1
                correct2 += 1
            elif key.strip() == ans[total][1]:
                correct2 += 1
            total += 1
        print("Accuracy with top 1 result: ", correct1 / total * 100, '%')
        print("Accuracy with top 2 result: ", correct2 / total * 100, '%')

In [6]:
train()
test()

Accuracy with top 1 result:  84.95400788436268 %
Accuracy with top 2 result:  94.87516425755584 %
